# Sumowanie zużycia zasobów
Notatnik w którym łączone jest zużycie procesora i ramu dla jednego znacznika czasowego.
Utworzone pliki lądują w folderze prepared.

In [ ]:
import pandas as pd
import os
import re
from datetime import datetime
import glob
entry_directory = "Raw"
prepared_directory = "Prepared"

files = glob.glob(f".\..\{entry_directory}" + '/**/*.csv', recursive=True)
file_count = len(files)
print(f"znaleziono {file_count} plików.")
i = 0
progress = 1
progress_interval = file_count*0.05
for file_path in files:
    if i > progress_interval*progress:
        progress_string = "#".rjust(progress, "#").ljust(20, '-')
        print(f"{progress_string} ({i}/{file_count})")
        progress += 1
    i+=1
    if re.search("\d\d_\d\d_\d\d\d\d_\d\d_\d\d_\d\d.csv$", file_path):
        
        if os.path.exists(file_path.replace(entry_directory, prepared_directory)):
            continue

        node_data_df = pd.read_csv(file_path, dtype = {'timestamp' : 'string', 'PID' : 'int', 'CPU': 'float64', 'RAM': 'float64'})
        node_data_df['timestamp'] = node_data_df['timestamp'].apply(lambda x: x if len(x.split(".")) > 1 else x + ".000000" )
        node_data_df['epoch'] = node_data_df['timestamp'] \
            .apply(lambda x: (datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f") - datetime(1970, 1, 1)).total_seconds())
        min_timestamp = node_data_df['epoch'].min()
        node_data_df['epoch'] = node_data_df['epoch'].apply(lambda x: x - min_timestamp)
        node_data_df.drop(['timestamp', 'PID'], axis=1, inplace=True)
        node_data_df = node_data_df.groupby("epoch").sum()
        root_dir = "/".join(file_path.split("\\")[:-1])
        if not os.path.exists(root_dir.replace(entry_directory, prepared_directory)):
            os.makedirs(root_dir.replace(entry_directory, prepared_directory))
        node_data_df.to_csv(file_path.replace(entry_directory, prepared_directory))
print("skończone.")